# Zoningtaxlots QAQC

### Objectives:
+ Combine `qc_versioncomparison.csv` with `qc_versioncomparisonnownullcount.csv`; sort by field name. (Resulting report will show __if the value changed to a different value or to/from a null value__).
+ Add a __difference__ column to `qc_frequencychanges.csv`; sort by field name.
+ Add two fields to the BBL diff report
    + Flag indicating that __lot intersects with a rezoning done since the last version__
    + Flag indicating that __the area of the lot (taken from DTM) has changed by more than +/- 10% since the last version__
+ Rename fields in BBL diff report for the fields showing the new data, using similar naming convention as used for previous data set, e.g., ZD1NEW.

In [45]:
import geopandas as gpd
import pandas as pd
import os
from sqlalchemy import create_engine
from pathlib import Path
import time
from shapely.wkb import dumps, loads
from shapely.wkt import loads as wkt_loads 

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 50)
print(time.strftime("%m/%d/%Y %H:%M:%S"))

02/03/2020 15:41:53


In [46]:
%load_ext dotenv
%dotenv ../.env

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [47]:
conn = create_engine(os.getenv('BUILD_ENGINE'))

In [48]:
recipe_conn = create_engine(os.getenv('RECIPE_ENGINE'))

In [49]:
# Reports the number of records that experienced a change in the value
query = '''
select field, count as diff_count, 
percent as diff_percent,  
        newnullcount, oldnullcount, 
        countnew as total_count_new, 
        countold as total_count_old, 
        (countnew-countold) as total_count_diff 
from(
    SELECT * FROM
        ztl_qc_versioncomparisoncount a 
    JOIN 
        ztl_qc_versioncomparisonnownullcount b
    USING (field)) c
JOIN
frequencychanges d
USING (field)
ORDER BY field;
'''
df_versioncomparison = pd.read_sql(sql=query, con=conn)
df_versioncomparison

,field,diff_count,diff_percent,newnullcount,oldnullcount,total_count_new,total_count_old,total_count_diff
0,commercialoverlay1,12.0,0.00,8,8,74945,74945,0
1,commercialoverlay2,0.0,0.00,0,0,166,166,0
2,limitedheightdistrict,0.0,0.00,0,0,3037,3037,0
3,specialdistrict1,0.0,0.00,0,0,101894,101891,3
4,specialdistrict2,0.0,0.00,0,0,80,80,0
5,specialdistrict3,0.0,0.00,0,0,0,0,0
6,zoningdistrict1,22.0,0.00,0,0,858403,858395,8
7,zoningdistrict2,119.0,0.01,53,53,19847,19830,17
8,zoningdistrict3,1.0,0.00,0,0,211,210,1
9,zoningdistrict4,0.0,0.00,0,0,13,13,0


In [50]:
# Reports the full zoning comarison table
query = '''
SELECT bblnew, bblprev, 
        zd1new, zd1prev, zd2new, zd2prev, zd3new, zd3prev, zd4new, zd4prev, 
        zmcnew, zmcprev, zmnnew, zmnprev, 
        co1new, co1prev, co2new, co2prev, 
        sd1new, sd1prev, sd2new, sd2prev, sd3new, sd3prev, 
        lhdnew, lhdprev, 
        inzonechange, mihflag, mihoption, 
        geom from bbldiffs;
'''
bbldiffs = gpd.GeoDataFrame.from_postgis(sql=query, con=conn)
bbldiffs

,bblnew,bblprev,zd1new,zd1prev,zd2new,zd2prev,zd3new,zd3prev,zd4new,zd4prev,zmcnew,zmcprev,zmnnew,zmnprev,co1new,co1prev,co2new,co2prev,sd1new,sd1prev,sd2new,sd2prev,sd3new,sd3prev,lhdnew,lhdprev,inzonechange,mihflag,mihoption,geom
0,5003110017,5003110017,R3-1,R3-1,None,R2,None,None,None,None,None,None,21A,21A,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-74.11577 40.62767, -74.11569 ..."
1,3044520235,3044520235,R7A,R7A,None,R6,None,None,None,None,None,None,17D,17D,C2-4,C2-4,None,None,None,None,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-73.87398 40.65560, -73.87414 ..."
2,3076080078,3076080078,R4,R4,R2,None,None,None,None,None,None,None,23B,23B,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-73.94945 40.62444, -73.94956 ..."
3,3044520670,3044520670,R7A,R7A,None,R6,None,None,None,None,None,None,17D,17D,C2-4,C2-4,None,None,None,None,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-73.87342 40.65481, -73.87363 ..."
4,5062180001,5062180001,R3-2,R3-2,None,R3X,None,None,None,None,None,None,33C,33C,None,None,None,None,SRD,SRD,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-74.17413 40.54115, -74.17591 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,5003880041,5003880041,R3X,R3X,None,None,None,None,None,None,None,None,21B,21B,C2-1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-74.13203 40.62559, -74.13247 ..."
159,5080430105,5080430105,R3X,R3X,R3A,None,None,None,None,None,None,None,35A,35A,C1-1,C1-1,None,None,SRD,SRD,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-74.24155 40.51188, -74.24152 ..."
160,5002300185,5002300185,R3A,R3-2,R3-2,R3A,None,None,None,None,None,None,21A,21A,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-74.12249 40.62779, -74.12253 ..."
161,4008100013,4008100013,R4-1,R4-1,R4,None,None,None,None,None,None,None,9C,9C,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-73.90390 40.77638, -73.90383 ..."


## DTM Comparison

In [55]:
version_old = '2020/01/07'
version_new = '2020/01/31'

In [56]:
# Reports lots that had an area change
query = f'''
with dtm_compare as (
    SELECT bbl, geom_new, geom_old, (case when geom_new = geom_old then 0 else 1 end) flag 
    FROM 
    (SELECT bbl, ST_Multi(ST_Union(f.wkb_geometry)) as geom_new 
        FROM dof_dtm."{version_new}" f GROUP BY bbl ) a
    JOIN 
    (SELECT bbl, ST_Multi(ST_Union(f.wkb_geometry)) as geom_old 
        FROM dof_dtm."{version_old}" f GROUP BY bbl ) b
    USING(bbl))
, changed as (
    SELECT *, (st_area(geom_new)-st_area(geom_old))/st_area(geom_old) as area_diff 
    FROM dtm_compare
    WHERE flag = 1)
SELECT * FROM changed WHERE area_diff > 0.1 OR area_diff < -0.1;
'''

In [57]:
bbl_areachange = gpd.GeoDataFrame.from_postgis(sql=query, con=recipe_conn, geom_col='geom_new')

In [58]:
bbl_areachange_new = bbl_areachange[['bbl', 'geom_new', 'area_diff']]

In [59]:
bbl_areachange_old = bbl_areachange[['bbl', 'geom_old', 'area_diff']]
bbl_areachange_old.loc[:, 'geom_old'] = bbl_areachange_old['geom_old'].apply(lambda x: wkt_loads(loads(x,  hex=True).wkt))
bbl_areachange_old=gpd.GeoDataFrame(bbl_areachange_old, geometry='geom_old')

In [60]:
from ipyleaflet import Map, basemaps, GeoData, basemap_to_tiles, LayersControl, FullScreenControl, Popup, Marker
from ipywidgets import HTML

m = Map(center=(40.730610, -73.935242), zoom=11)

dark_matter_layer = basemap_to_tiles(basemaps.CartoDB.DarkMatter, close_popup_on_click=True)
m.add_layer(dark_matter_layer)

In [61]:
new = GeoData(geo_dataframe = bbl_areachange_new, 
              style={'color': 'green', 'opacity':10, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.3},
              name = 'new')

old = GeoData(geo_dataframe = bbl_areachange_old, 
              style={'color': 'red', 'opacity':10, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.3}, 
              name = 'old')

In [62]:
m.add_layer(new)
m.add_layer(old)
m.add_control(LayersControl())
m.add_control(FullScreenControl())

In [63]:
for i in range(bbl_areachange_new.shape[0]):
    center = (bbl_areachange_new.loc[i, 'geom_new'].centroid.y,
              bbl_areachange_new.loc[i, 'geom_new'].centroid.x)
    bbl = str(list(bbl_areachange_new['bbl'])[i])
    area_change = round(list(bbl_areachange_new['area_diff'])[i]*100, 2)
    marker = Marker(location=center)
    m.add_layer(marker)
    marker.popup = HTML(value=f'''<a href=https://zola.planning.nyc.gov/bbl/{bbl}> {bbl} </a>
                                  <p> area change: {area_change}% </p>''')

In [64]:
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …